## Importing 'DDW-C19-0000.XLSX'(Language Data)  and 'DDW-0000C-08.xlsx'(Census Data according to age) file using pandas

In [20]:
import pandas as pd
data = pd.read_excel('INPUT/DDW-C19-0000.XLSX')
census_data = pd.read_excel('INPUT/DDW-0000C-08.xlsx')

### Getting the unique list of states/UTs from the census data to check whether they are similar to the ones obtained from the language data

In [21]:
name_lst = list(census_data['Unnamed: 3'][6:].unique())
name_lst1 = name_lst[:]
for i in range(len(name_lst)):
    if name_lst[i] != "INDIA":
        name_lst[i] = name_lst[i].split("- ")[1]

### Checking if both have the same states with same sequence

In [22]:
state_lang = list(data['Unnamed: 2'][5:869].unique())
yes = 0
for i in range(len(state_lang)):
    if name_lst[i] == state_lang[i]:
        yes = yes+1
print(yes)

36


### Getting the names of the literacy groups from the language data according to literacy

In [23]:
lit_grp = list(data['Unnamed: 4'][5:869].unique())
lit_grp = lit_grp[1:]
lit_grp.remove('Literate')
lit_grp

['Illiterate',
 'Literate but below primary',
 'Primary but below middle',
 'Middle but below matric/secondary',
 'Matric/Secondary but below graduate',
 'Graduate and above']

### Getting the unique names of literacy groups from the census data
### We only need groups from 'Below' which indicates 'Literate but below primary' to 'Graduate & above'

In [24]:
lit_grp_cen = list(census_data.iloc[1].unique())
lit_grp_cen = lit_grp_cen[5:13]
lit_grp_cen

['Below',
 'Primary',
 'Middle',
 'Matric/Secondary',
 'Higher secondary/Intermediate',
 'Non-technical diploma or',
 'Technical diploma or certificate',
 'Graduate & above']

### Getting the column names of the columns consisting the values for the literacy groups found above

In [ ]:
lit_grp_cen_ind = []
for i in range(len(lit_grp_cen)):
    lit_grp_cen_ind.append(list(census_data.columns[
    (census_data == lit_grp_cen[i])
    .any(axis=0)
    ])[0])

### Adding column name of 'Illiterate' to the list

In [ ]:
lit_grp_cen_ind.insert(0,'Unnamed: 9')

### Getting the column names of the columns consisting the values for the literacy groups for males and females

In [ ]:
male_lit_grp_cen_ind = [0]* len(lit_grp_cen_ind)
female_lit_grp_cen_ind = [0]* len(lit_grp_cen_ind)
for i in range(len(lit_grp_cen_ind)):
    male_lit_grp_cen_ind[i] = lit_grp_cen_ind[i].split(" ")[0] + " " +str(int(lit_grp_cen_ind[i].split(" ")[1]) + 1)
    female_lit_grp_cen_ind[i] = lit_grp_cen_ind[i].split(" ")[0] + " " + str(int(lit_grp_cen_ind[i].split(" ")[1]) + 2)

### Getting the total population of males for each state/UT for each literacy group in a list of lists

In [ ]:
male_lit_grp_tot_cen = [[0]*len(lit_grp)]*len(name_lst1)
for i in range(len(name_lst)):
    temp = [0] * len(lit_grp)
    for j in range(len(lit_grp)):
        if j in range(0,4):
            temp[j] = census_data[male_lit_grp_cen_ind[j]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))]
        elif j == 4:
            temp[j] = (census_data[male_lit_grp_cen_ind[j]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))] + 
            census_data[male_lit_grp_cen_ind[j+1]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))] + 
            census_data[male_lit_grp_cen_ind[j+2]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))] + 
            census_data[male_lit_grp_cen_ind[j+3]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))])
        else:
            temp[j] = census_data[male_lit_grp_cen_ind[j+3]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))]
    male_lit_grp_tot_cen[i] = temp

### Getting the total population of females for each state/UT for each literacy group in a list of lists

In [ ]:
female_lit_grp_tot_cen = [[0]*len(lit_grp)]*len(name_lst1)
for i in range(len(name_lst)):
    temp = [0] * len(lit_grp)
    for j in range(len(lit_grp)):
        if j in range(0,4):
            temp[j] = census_data[female_lit_grp_cen_ind[j]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))]
        elif j == 4:
            temp[j] = (census_data[female_lit_grp_cen_ind[j]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))] + 
            census_data[female_lit_grp_cen_ind[j+1]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))] + 
            census_data[female_lit_grp_cen_ind[j+2]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))] + 
            census_data[female_lit_grp_cen_ind[j+3]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))])
        else:
            temp[j] = census_data[female_lit_grp_cen_ind[j+3]][(list(census_data['Unnamed: 3']).index(name_lst1[i]))]
    female_lit_grp_tot_cen[i] = temp

### Getting the ratio of males speaking 3 languages or more, males speaking 2 languages, males speaking only one language with total male population(found above) for each state for each literacy group

In [ ]:
male_lit_grp_3lang_ratio = [[0]*len(lit_grp)]*len(state_lang)
male_lit_grp_2lang_ratio = [[0]*len(lit_grp)]*len(state_lang)
male_lit_grp_1lang_ratio = [[0]*len(lit_grp)]*len(state_lang) 
for i in range(len(state_lang)):
    temp3 = [0]*len(lit_grp)
    temp2 = [0]*len(lit_grp)
    temp1 = [0]*len(lit_grp)
    for j in range(len(lit_grp)):
        if lit_grp[j] == 'Illiterate':
            temp3[j] = (data['Unnamed: 9'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+1])
            temp2[j] = (data['Unnamed: 6'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+1] - temp3[j])
            temp1[j] = (male_lit_grp_tot_cen[i][j] - temp2[j] - temp3[j])
            temp3[j] = temp3[j] / male_lit_grp_tot_cen[i][j]
            temp2[j] = temp2[j] / male_lit_grp_tot_cen[i][j]
            temp1[j] = temp1[j] / male_lit_grp_tot_cen[i][j]
        else:
            temp3[j] = (data['Unnamed: 9'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+2])
            temp2[j] = (data['Unnamed: 6'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+2] - temp3[j])
            temp1[j] = (male_lit_grp_tot_cen[i][j] - temp2[j] - temp3[j])
            temp3[j] = temp3[j] / male_lit_grp_tot_cen[i][j]
            temp2[j] = temp2[j] / male_lit_grp_tot_cen[i][j]
            temp1[j] = temp1[j] / male_lit_grp_tot_cen[i][j]
    male_lit_grp_3lang_ratio[i] = temp3
    male_lit_grp_2lang_ratio[i] = temp2
    male_lit_grp_1lang_ratio[i] = temp1

### Getting the ratio of females speaking 3 languages or more, females speaking 2 languages, females speaking only one language with total female population(found above) for each state for each literacy group

In [ ]:
female_lit_grp_3lang_ratio = [[0]*len(lit_grp)]*len(state_lang)
female_lit_grp_2lang_ratio = [[0]*len(lit_grp)]*len(state_lang)
female_lit_grp_1lang_ratio = [[0]*len(lit_grp)]*len(state_lang) 
for i in range(len(state_lang)):
    temp3 = [0]*len(lit_grp)
    temp2 = [0]*len(lit_grp)
    temp1 = [0]*len(lit_grp)
    for j in range(len(lit_grp)):
        if lit_grp[j] == 'Illiterate':
            temp3[j] = (data['Unnamed: 10'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+1])
            temp2[j] = (data['Unnamed: 7'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+1] - temp3[j])
            temp1[j] = (female_lit_grp_tot_cen[i][j] - temp2[j] - temp3[j])
            temp3[j] = temp3[j] / female_lit_grp_tot_cen[i][j]
            temp2[j] = temp2[j] / female_lit_grp_tot_cen[i][j]
            temp1[j] = temp1[j] / female_lit_grp_tot_cen[i][j]
        else:
            temp3[j] = (data['Unnamed: 10'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+2])
            temp2[j] = (data['Unnamed: 7'][(list(data['Unnamed: 2']).index(state_lang[i]))+j+2] - temp3[j])
            temp1[j] = (female_lit_grp_tot_cen[i][j] - temp2[j] - temp3[j])
            temp3[j] = temp3[j] / female_lit_grp_tot_cen[i][j]
            temp2[j] = temp2[j] / female_lit_grp_tot_cen[i][j]
            temp1[j] = temp1[j] / female_lit_grp_tot_cen[i][j]
    female_lit_grp_3lang_ratio[i] = temp3
    female_lit_grp_2lang_ratio[i] = temp2
    female_lit_grp_1lang_ratio[i] = temp1

### Getting the highest ratio from all the literacy groups for each state and the name of that group for males

In [ ]:
male_ratio_3max = [0] * len(state_lang)
male_ratio_2max = [0] * len(state_lang)
male_ratio_1max = [0] * len(state_lang)
male_ratio_3max_litgrp = [0] * len(state_lang)
male_ratio_2max_litgrp = [0] * len(state_lang)
male_ratio_1max_litgrp = [0] * len(state_lang)
for i in range(len(name_lst)):
    male_ratio_3max[i] = max(male_lit_grp_3lang_ratio[i])
    male_ratio_2max[i] = max(male_lit_grp_2lang_ratio[i])
    male_ratio_1max[i] = max(male_lit_grp_1lang_ratio[i])
    male_ratio_3max_litgrp[i] = lit_grp[male_lit_grp_3lang_ratio[i].index(male_ratio_3max[i])]
    male_ratio_2max_litgrp[i] = lit_grp[male_lit_grp_2lang_ratio[i].index(male_ratio_2max[i])]
    male_ratio_1max_litgrp[i] = lit_grp[male_lit_grp_1lang_ratio[i].index(male_ratio_1max[i])]

### Getting the highest ratio from all the literacy groups for each state and the name of that group for females

In [ ]:
female_ratio_3max = [0] * len(state_lang)
female_ratio_2max = [0] * len(state_lang)
female_ratio_1max = [0] * len(state_lang)
female_ratio_3max_litgrp = [0] * len(state_lang)
female_ratio_2max_litgrp = [0] * len(state_lang)
female_ratio_1max_litgrp = [0] * len(state_lang)
for i in range(len(name_lst)):
    female_ratio_3max[i] = max(female_lit_grp_3lang_ratio[i])
    female_ratio_2max[i] = max(female_lit_grp_2lang_ratio[i])
    female_ratio_1max[i] = max(female_lit_grp_1lang_ratio[i])
    female_ratio_3max_litgrp[i] = lit_grp[female_lit_grp_3lang_ratio[i].index(female_ratio_3max[i])]
    female_ratio_2max_litgrp[i] = lit_grp[female_lit_grp_2lang_ratio[i].index(female_ratio_2max[i])]
    female_ratio_1max_litgrp[i] = lit_grp[female_lit_grp_1lang_ratio[i].index(female_ratio_1max[i])]

### Getting the codes of the states

In [ ]:
state_code = [0] * len(name_lst)
for i in range(len(name_lst)):
    state_code[i] = data['C-19 POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX'][list(data['Unnamed: 2']).index(name_lst[i])]

### Exporting the state codes, state names, literacy groups, ratios into dataframes

In [ ]:
data_final = {'state-code': state_code, 'state/ut': state_lang,'literacy-group-males': male_ratio_3max_litgrp,'ratio-males': male_ratio_3max,'literacy-group-females': female_ratio_3max_litgrp,'ratio-females': male_ratio_3max}
literacy_gender_3 = pd.DataFrame(data_final)
data_final = {'state-code': state_code, 'state/ut': state_lang,'literacy-group-males': male_ratio_2max_litgrp,'ratio-males': male_ratio_2max,'literacy-group-females': female_ratio_2max_litgrp,'ratio-females': male_ratio_2max}
literacy_gender_2 = pd.DataFrame(data_final)
data_final = {'state-code': state_code, 'state/ut': state_lang,'literacy-group-males': male_ratio_1max_litgrp,'ratio-males': male_ratio_1max,'literacy-group-females': female_ratio_1max_litgrp,'ratio-females': male_ratio_1max}
literacy_gender_1 = pd.DataFrame(data_final)

### Exporting dataframes into 'literacy-gender-{part}.csv'

In [ ]:
literacy_gender_3.to_csv('OUTPUT/literacy-gender-a.csv', index = False)
literacy_gender_2.to_csv('OUTPUT/literacy-gender-b.csv', index = False)
literacy_gender_1.to_csv('OUTPUT/literacy-gender-c.csv', index = False)